In [11]:
import numpy as np

def barrier_option_pricing():
    S: float      # 標的資產當前價格
    X: float      # 履約價
    r: float      # 無風險利率 (年化)
    sigma: float  # 波動率 (年化)
    T: float      # 到期時間 (年)
    H: float      # 障礙水平
    option_type: str # 'call' or 'put'
    barrier_type: str # 'up_in', 'down_in', 'up_out', 'down_out'
    n_sims: int   # 模擬路徑數量
    n_steps: int   # 每個路徑的步數

    

In [12]:
"""
使用蒙地卡羅模擬評價障礙選擇權。
"""

dt = T / n_steps

# 創建隨機數矩陣，用於模擬多條路徑
# 每一行代表一條路徑，每一列代表一個時間步
# 使用 np.random.standard_normal 創建標準常態分佈隨機數
random_matrix = np.random.standard_normal((n_sims, n_steps))

# 計算資產價格路徑
# S_t+1 = S_t * exp((r - 0.5 * sigma^2) * dt + sigma * sqrt(dt) * Z)
# 這裡我們一次性計算所有時間步的價格變化因子，效率更高
price_paths = np.zeros((n_sims, n_steps + 1))
price_paths[:, 0] = S

for i in range(n_steps):
    drift = (r - 0.5 * sigma**2) * dt
    shock = sigma * np.sqrt(dt) * random_matrix[:, i]
    price_paths[:, i+1] = price_paths[:, i] * np.exp(drift + shock)

# 初始化最終支付數組
payoffs = np.zeros(n_sims)

# 檢查是否觸及障礙
# 這裡是關鍵：檢查每一條路徑在任何一個時間點是否觸及或穿過障礙
# axis=1 代表在每一行（即每一條路徑）上檢查
has_hit_barrier = np.any(price_paths >= H, axis=1) if 'up' in barrier_type else np.any(price_paths <= H, axis=1)

# 根據選擇權類型計算支付
if 'in' in barrier_type:
    # Knock-in 選擇權: 只有在觸及障礙時才有效

    # 找到所有觸及障礙的路徑
    hit_paths = price_paths[has_hit_barrier]

    # 檢查這些路徑在到期日的最終價格
    final_prices = hit_paths[:, -1]

    if option_type == 'call':
        payoffs[has_hit_barrier] = np.maximum(final_prices - X, 0)
    else: # put
        payoffs[has_hit_barrier] = np.maximum(X - final_prices, 0)

elif 'out' in barrier_type:
    # Knock-out 選擇權: 如果觸及障礙，則失效 (支付為0)

    # 找到所有未觸及障礙的路徑
    not_hit_paths = price_paths[~has_hit_barrier]

    # 檢查這些路徑在到期日的最終價格
    final_prices = not_hit_paths[:, -1]

    if option_type == 'call':
        payoffs[~has_hit_barrier] = np.maximum(final_prices - X, 0)
    else: # put
        payoffs[~has_hit_barrier] = np.maximum(X - final_prices, 0)

# 計算平均支付並折現回現值
present_value = np.mean(payoffs) * np.exp(-r * T)

return present_value

# --- 範例用法 ---

if __name__ == "__main__":
# 輸入參數
    S = 100.0        # 標的價格
    X = 100.0        # 履約價
    r = 0.05         # 無風險利率 (5%)
    sigma = 0.20     # 波動率 (20%)
    T = 1.0          # 到期時間 (1年)
    H = 120.0        # 向上障礙

    # 模擬參數
    n_sims = 100000  # 模擬路徑數量
    n_steps = 252    # 每個路徑的步數 (約為一年中的交易日)

    # 評價一個向上觸及式買權 (Up-and-in Call)
    # 只有當價格超過 120 時，這個買權才有效
    up_in_call_price = barrier_option_pricing(
        S, X, r, sigma, T, H,
        option_type='call',
        barrier_type='up_in',
        n_sims=n_sims,
        n_steps=n_steps
    )
    print(f"向上觸及式買權 (Up-and-in Call) 價格: {up_in_call_price:.4f}")

    # 評價一個向上出局式買權 (Up-and-out Call)
    # 當價格超過 120 時，這個買權立即失效
    up_out_call_price = barrier_option_pricing(
        S, X, r, sigma, T, H,
        option_type='call',
        barrier_type='up_out',
        n_sims=n_sims,
        n_steps=n_steps
    )
    print(f"向上出局式買權 (Up-and-out Call) 價格: {up_out_call_price:.4f}")

    # 評價一個向下出局式賣權 (Down-and-out Put)
    H_down = 80.0 # 向下障礙
    down_out_put_price = barrier_option_pricing(
        S, X, r, sigma, T, H_down,
        option_type='put',
        barrier_type='down_out',
        n_sims=n_sims,
        n_steps=n_steps
    )
    print(f"向下出局式賣權 (Down-and-out Put) 價格: {down_out_put_price:.4f}")

NameError: name 'barrier_type' is not defined